In [0]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
import torch

In [3]:
accelerator

'cu80'

In [0]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import nltk
import random
import numpy as np
from collections import Counter, OrderedDict
import re
from copy import deepcopy
import pandas as pd
import numpy as np
from torch.nn.utils.rnn import PackedSequence,pack_padded_sequence
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

flatten = lambda l: [item for sublist in l for item in sublist]

In [0]:
# 安装 PyDrive 操作库，该操作每个 notebook 只需要执行一次
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 授权登录，仅第一次的时候会鉴权
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [6]:
# 列出根目录的所有文件
# "q" 查询条件教程详见：https://developers.google.com/drive/v2/web/search-parameters
file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s, mimeType: %s' % (file1['title'], file1['id'], file1["mimeType"]))

title: data, id: 16zfnt8Cnuznzwj2RHGrxvyfyA6iQPRXx, mimeType: application/vnd.google-apps.folder
title: Colab Notebooks, id: 1DDn16_H4dL75WfkYDpWNxRzqJkH6uvsk, mimeType: application/vnd.google-apps.folder
title: 02 - email-templates.png, id: 0BzICsNHMK-GFdnZhNFBoVXdobkhBa3h2eUtwbXY1ek15Sk53, mimeType: image/png
title: ATEC-Alipay, id: 1UyfXNCoPHh73E4jO-pgombXQShfYkzAVgGfSUnU7OAE, mimeType: application/vnd.google-apps.document
title: The Annotated _Attention is All You Need_.ipynb, id: 1zIkaTyyRtMY0qRTuUxe9J10xR3S9J5Xc, mimeType: application/vnd.google.colab
title: Copy of mixture_of_softmaxes.ipynb, id: 1HiJvBimCdWvFP7SoLs9bWNR-JbCQSRsG, mimeType: application/vnd.google.colab
title: Copy of Hello, Colaboratory, id: 0BzICsNHMK-GFcFBrQkV2UXpMVGM, mimeType: application/vnd.google.colab
title: Screen Shot 2017-10-12 at 9.35.30 PM.png, id: 0BzICsNHMK-GFMjFrZzQwOHZvYVRiMWgtVlVfSlVRU1FYZmdn, mimeType: image/png
title: 2017 Calendar, id: 1jw6VChc9SGayYuK5Zmy2PjaiwWM0tzGFrPG9eatsSNw, mimeType: 

In [7]:
# '目录 id' in parents
file_list = drive.ListFile({'q': "'16zfnt8Cnuznzwj2RHGrxvyfyA6iQPRXx' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s, mimeType: %s' % (file1['title'], file1['id'], file1["mimeType"]))

title: atec_nlp_sim_train.csv, id: 1py3JAFOLOH2lM-lOqYyP_LfqaImlOqL_, mimeType: text/csv


In [0]:
file = drive.CreateFile({'id': "1py3JAFOLOH2lM-lOqYyP_LfqaImlOqL_"}) 
#这里的下载操作只是缓存，不会在你的Google Drive 目录下多下载一个文件
file.GetContentFile('data.csv', "text/csv") 

In [0]:
df = pd.read_csv('data.csv', sep='\t', names=['number', "sen1", "sen2", "label"], skipinitialspace=True)

In [10]:
df.head(2)

,number,sen1,sen2,label
0,1,﻿怎么更改花呗手机号码,我的花呗是以前的手机号码，怎么更改成现在的支付宝的号码手机号,1
1,2,也开不了花呗，就这样了？完事了,真的嘛？就是花呗付款,0


In [0]:
def normalize_string(s):
    s = re.sub(r"\s+", r" ", s).strip()
    return s

In [12]:
%%time

X1_r = list(map(normalize_string, df.sen1.tolist()))
X2_r = list(map(normalize_string, df.sen2.tolist()))
y_r = df.label.tolist()
print(len(X1_r), len(X2_r), len(y_r))
print(X1_r[0], "@@@@", X2_r[0], "@@@@",y_r[0])

39346 39346 39346
﻿怎么更改花呗手机号码 @@@@ 我的花呗是以前的手机号码，怎么更改成现在的支付宝的号码手机号 @@@@ 1
CPU times: user 126 ms, sys: 3 ms, total: 129 ms
Wall time: 131 ms


In [13]:
vocab = list(set(flatten(X1_r + X2_r)))
len(vocab)

1716

In [0]:
source2index = {'<PAD>':0,'<UNK>':1,'<s>':2,'</s>':3}
for vo in vocab:
    if vo not in source2index.keys():
        source2index[vo]=len(source2index)
index2source = {v:k for k,v in source2index.items()}

In [15]:
index2source[source2index[","]]

','

In [0]:
EPOCH=50
BATCH_SIZE = 64
EMBEDDING_SIZE = 128
HIDDEN_SIZE = 256
LR = 0.01
DECODER_LEARNING_RATIO=5.0
RESCHEDULED=False

In [0]:
USE_CUDA = torch.cuda.is_available()
FloatTensor = torch.cuda.FloatTensor if USE_CUDA else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if USE_CUDA else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if USE_CUDA else torch.ByteTensor

In [18]:
USE_CUDA

True

In [19]:
LongTensor

torch.cuda.LongTensor

In [0]:
def getBatch(batch_size,train_data):
    random.shuffle(train_data)
    sindex=0
    eindex=batch_size
    while eindex < len(train_data):
        batch = train_data[sindex:eindex]
        temp = eindex
        eindex = eindex+batch_size
        sindex = temp
        yield batch
    
    if eindex >= len(train_data):
        batch = train_data[sindex:]
        yield batch

In [0]:
def prepare_sequence(seq, to_index):
    idxs = list(map(lambda w: to_index[w] if w in to_index.keys() else to_index["<unk>"], seq))
    return LongTensor(idxs)

In [106]:
%%time
X1_p, X2_p = [],[]
ta_p = []

for s1, s2, ta in zip(X1_r, X2_r, y_r):
    X1_p.append(prepare_sequence(s1,source2index).view(1,-1))
    X2_p.append(prepare_sequence(s2,source2index).view(1,-1))
    ta_p.append(ta)
    
data = list(zip(X1_p, X2_p, ta_p))
random.shuffle(data)
train_data = data[:30000]
test_data = data[30000:]

print(train_data[0])

(
  413  1184   958  1025   309   913   809  1613   227   510  1025  1329   197
[torch.cuda.LongTensor of size 1x13 (GPU 0)]
, 
  895  1690   413  1184   172  1050   510  1349  1302  1329  1613   197
[torch.cuda.LongTensor of size 1x12 (GPU 0)]
, 0)
CPU times: user 3.3 s, sys: 1.27 s, total: 4.57 s
Wall time: 4.57 s


In [0]:
def pad_to_batch(batch,x_to_ix):
    
    sorted_batch =  sorted(batch, key=lambda b:b[0].size(1),reverse=True) # sort by len
    
    x1, x2, y = list(zip(*sorted_batch))
    max_x1 = max([s.size(1) for s in x1])
    max_x2 = max([s.size(1) for s in x2])
    
    x1_p, x2_p, y_p = [],[],[]
    for i in range(len(batch)):
      
        if x1[i].size(1)<max_x1:
            x1_p.append(torch.cat( [ Variable(x1[i]),Variable(LongTensor([x_to_ix['<PAD>']]*(max_x1-x1[i].size(1)))).view(1,-1) ],1))
        else:
            x1_p.append(Variable(x1[i]))
        
        if x2[i].size(1)<max_x2:
            #v = Variable(LongTensor([x_to_ix['<PAD>']]*(max_x2-x2[i].size(1)))).view(1,-1)
            #print(torch.cat((Variable(x2[i]), v), 1))
            x2_p.append(torch.cat( [ Variable(x2[i]),Variable(LongTensor([x_to_ix['<PAD>']]*(max_x2-x2[i].size(1)))).view(1,-1) ],1))
        else:
            x2_p.append(Variable(x2[i]))
        
    x1_var = torch.cat(x1_p, 0)
    x2_var = torch.cat(x2_p, 0)
    target_var = Variable(LongTensor(y))
    x1_len = [list(map(lambda s: s ==0, t.data)).count(False) for t in x1_var]
    x2_len = [list(map(lambda s: s ==0, t.data)).count(False) for t in x2_var]
    
    return x1_var, x2_var, target_var, x1_len, x2_len

In [0]:
batch = next(getBatch(BATCH_SIZE,train_data))

In [0]:
pbatch = pad_to_batch(batch,source2index)

In [69]:
pbatch[0]

Variable containing:
   29   344   289  ...   1341   605  1698
  866  1634    98  ...    106  1321     0
 1398  1179    98  ...      0     0     0
       ...          ⋱          ...       
 1554  1184   482  ...      0     0     0
  413  1184   740  ...      0     0     0
  413  1184   703  ...      0     0     0
[torch.cuda.LongTensor of size 64x29 (GPU 0)]

In [0]:
class EncoderV(nn.Module):
    def __init__(self, input_size, embedding_size,hidden_size, n_layers=1,bidirec=False):
        super(EncoderV, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_size, embedding_size)
        
        if bidirec:
            self.n_direction = 2 
            self.gru = nn.GRU(embedding_size, hidden_size, n_layers, batch_first=True,bidirectional=True)
        else:
            self.n_direction = 1
            self.gru = nn.GRU(embedding_size, hidden_size, n_layers, batch_first=True)
    
    def init_hidden(self,inputs):  # input.size(0) = batch_size
        hidden = Variable(torch.zeros(self.n_layers*self.n_direction,inputs.size(0),self.hidden_size))
        return hidden.cuda() if USE_CUDA else hidden
    
    def init_weight(self):
        self.embedding.weight = nn.init.xavier_uniform(self.embedding.weight)
        self.gru.weight_hh_l0 = nn.init.xavier_uniform(self.gru.weight_hh_l0)
        self.gru.weight_ih_l0 = nn.init.xavier_uniform(self.gru.weight_ih_l0)
    
    def forward(self, x, x_len):
        """
        sequence -> sort -> pad and pack ->process using RNN -> unpack ->unsort

        :param x: Variable
        :param x_len: numpy list
        :return:
        """
        
        hidden = self.init_hidden(x)
        
        """sort"""
        x_sort_idx = np.argsort(x_len)[::-1]
        x_unsort_idx = LongTensor(np.argsort(x_sort_idx))
        x_len = np.array(x_len)[x_sort_idx]
        x = x[LongTensor(x_sort_idx.copy())]
      
        embedded = self.embedding(x)
        
        """pack"""
        x_emb_p = torch.nn.utils.rnn.pack_padded_sequence(embedded, x_len, batch_first=True)
                
        """process using RNN"""
        out_pack, ht = self.gru(x_emb_p, hidden)
        
        """unsort: h"""
        ht = torch.transpose(ht, 0, 1)[
            x_unsort_idx]  # (num_layers * num_directions, batch, hidden_size) -> (batch, ...)
        ht = torch.transpose(ht, 0, 1)

        #print("ht", ht.shape)
        if self.n_layers>1:
            if self.n_direction==2:
                ht = ht[-2:]
                return out_pack, torch.cat((ht[0], ht[1]),1)
            else:
                ht = ht[-1]
                return out_pack, ht

     

In [0]:
class Model(nn.Module):
    
    def __init__(self, encoder1, encoder2, hidden_size):
        
        super(Model, self).__init__()
        self.encoder1 = encoder1
        
        self.fc1 = nn.Linear(hidden_size*4, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, 2)
        
    def init_weight(self):
        
        self.encoder1.init_weight()
        #self.fc1.weight = nn.init.xavier_uniform(self.fc1.weight)
        #self.fc2.weight = nn.init.xavier_uniform(self.fc2.weight)
        
    def forward(self, sen1, sen2, sen1_lengths, sen2_lengths):
             
        outputs_1, hidden_c1 = encoder1(sen1,sen1_lengths)
        outputs_2, hidden_c2 = encoder1(sen2,sen2_lengths)
        
        hidden = torch.cat((hidden_c1, hidden_c2), 1).squeeze(1)  # batch * 2hidden
        out = self.fc1(hidden)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [0]:
encoder1 = EncoderV(len(source2index),EMBEDDING_SIZE,HIDDEN_SIZE,2,True)
if USE_CUDA:
    encoder1 = encoder1.cuda()
    
model = Model(encoder1, encoder1, HIDDEN_SIZE)
model.init_weight()

if USE_CUDA:
    model = model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=LR)

In [53]:
model.parameters

<bound method Module.parameters of Model(
  (encoder1): EncoderV(
    (embedding): Embedding(1720, 128)
    (gru): GRU(128, 256, num_layers=2, batch_first=True, bidirectional=True)
  )
  (fc1): Linear(in_features=1024, out_features=256)
  (relu): ReLU()
  (fc2): Linear(in_features=256, out_features=2)
)>

In [0]:
for i,p in enumerate(model.parameters()):
    if p.requires_grad:
        print(i, ":", p.shape)

In [54]:
list(model.parameters())[17]

Parameter containing:
-2.4837e-02 -3.9009e-03  5.5835e-03  ...  -1.4489e-02 -2.0116e-02  1.9349e-02
 5.9822e-03 -5.6138e-04  2.2489e-02  ...   1.2735e-02 -1.2461e-03 -1.8798e-02
-2.1907e-02 -1.5935e-02  2.3978e-02  ...   5.8109e-03 -3.6775e-03 -1.6479e-02
                ...                   ⋱                   ...                
-1.4632e-02 -3.1242e-02  2.4850e-02  ...   1.4655e-02  2.0924e-02  4.6784e-03
 3.0224e-02  1.5460e-02 -2.3572e-02  ...   1.2580e-02  1.5037e-02  2.8139e-03
-2.1273e-02 -9.3140e-03 -1.7181e-02  ...   2.2980e-02  2.7514e-02  2.6040e-02
[torch.cuda.FloatTensor of size 256x1024 (GPU 0)]

In [55]:
list(model.parameters())[5]

Parameter containing:
 5.0677e-02 -4.0495e-03 -1.5734e-02  ...   6.1833e-02  1.9878e-03  3.9480e-02
-6.0898e-02  4.5098e-02 -4.0118e-02  ...  -5.6398e-02 -2.4148e-02  7.2464e-03
-4.6499e-02  5.7653e-02  3.6538e-02  ...   1.7100e-02 -5.4041e-03 -3.7763e-02
                ...                   ⋱                   ...                
 5.4879e-03  5.3690e-02  5.9184e-02  ...   2.5614e-02  5.1558e-03  1.0438e-03
-6.1255e-03  6.0199e-03  2.8708e-02  ...  -4.9139e-02  2.0534e-02 -5.2865e-02
-3.3467e-02 -8.7433e-03  1.7369e-02  ...   5.8226e-02  2.8482e-02  1.7205e-02
[torch.cuda.FloatTensor of size 768x128 (GPU 0)]

In [56]:
### test 
for i,batch in enumerate(getBatch(BATCH_SIZE,train_data)):
    
    # sen1 = batch * l
    sen1, sen2, targets,sen1_lengths, sen2_lengths = pad_to_batch(batch,source2index)
    print("sen1 size: ", sen1.shape)
    #input_masks = torch.cat([Variable(ByteTensor(tuple(map(lambda s: s ==0, t.data)))) for t in sen1]).view(sen1.size(0),-1)
    model.zero_grad()
    optimizer.zero_grad()
    
    # Forward + Backward + Optimize
    outputs = model(sen1, sen2, sen1_lengths, sen2_lengths)
    print('outputs', outputs.size())
    print('targets', targets.size())
    loss = criterion(outputs, targets)
    print('loss', loss)
    loss.backward()
    optimizer.step()
    
    break

sen1 size:  torch.Size([64, 44])
outputs torch.Size([64, 2])
targets torch.Size([64])
loss Variable containing:
 0.7057
[torch.cuda.FloatTensor of size 1 (GPU 0)]



In [0]:
total_step = round(len(train_data) / BATCH_SIZE)

for epoch in range(EPOCH):
    
    total_loss = []
    
    for i,batch in enumerate(getBatch(BATCH_SIZE,train_data)):
        sen1, sen2, targets,sen1_lengths, sen2_lengths = pad_to_batch(batch,source2index)
        model.zero_grad()
        optimizer.zero_grad()

        # Forward + Backward + Optimize
        outputs = model(sen1, sen2, sen1_lengths, sen2_lengths)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        total_loss.append(loss.data[0])

        if (i+1) % 20 == 0:
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                       .format(epoch+1, EPOCH, i+1, total_step, loss.data[0]))
     
    print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, EPOCH, np.mean(total_loss)))
    
    if RESCHEDULED == False and epoch  == EPOCH//2:
        LR *= 0.01
        optimizer = optim.Adam(model.parameters(), lr=LR)
        RESCHEDULED = True

Epoch [1/50], Step [20/469], Loss: 0.4953
Epoch [1/50], Step [40/469], Loss: 0.5467
Epoch [1/50], Step [60/469], Loss: 0.5816
Epoch [1/50], Step [80/469], Loss: 0.5139
Epoch [1/50], Step [100/469], Loss: 0.5113
Epoch [1/50], Step [120/469], Loss: 0.5806
Epoch [1/50], Step [140/469], Loss: 0.4849
Epoch [1/50], Step [160/469], Loss: 0.5790
Epoch [1/50], Step [180/469], Loss: 0.5041
Epoch [1/50], Step [200/469], Loss: 0.5422
Epoch [1/50], Step [220/469], Loss: 0.5795
Epoch [1/50], Step [240/469], Loss: 0.4388
Epoch [1/50], Step [260/469], Loss: 0.5791
Epoch [1/50], Step [280/469], Loss: 0.5041
Epoch [1/50], Step [300/469], Loss: 0.4556
Epoch [1/50], Step [320/469], Loss: 0.5174
Epoch [1/50], Step [340/469], Loss: 0.3957
Epoch [1/50], Step [360/469], Loss: 0.5238
Epoch [1/50], Step [380/469], Loss: 0.5586
Epoch [1/50], Step [400/469], Loss: 0.5081
Epoch [1/50], Step [420/469], Loss: 0.4482
Epoch [1/50], Step [440/469], Loss: 0.5306
Epoch [1/50], Step [460/469], Loss: 0.5627
Epoch [1/50], L

Epoch [3/50], Step [420/469], Loss: 0.5253
Epoch [3/50], Step [440/469], Loss: 0.5645
Epoch [3/50], Step [460/469], Loss: 0.3679
Epoch [3/50], Loss: 0.5240
Epoch [4/50], Step [20/469], Loss: 0.5254
Epoch [4/50], Step [40/469], Loss: 0.5050
Epoch [4/50], Step [60/469], Loss: 0.4858
Epoch [4/50], Step [80/469], Loss: 0.5848
Epoch [4/50], Step [100/469], Loss: 0.4139
Epoch [4/50], Step [120/469], Loss: 0.5449
Epoch [4/50], Step [140/469], Loss: 0.5642
Epoch [4/50], Step [160/469], Loss: 0.6197
Epoch [4/50], Step [180/469], Loss: 0.5456
Epoch [4/50], Step [200/469], Loss: 0.4408
Epoch [4/50], Step [220/469], Loss: 0.5903
Epoch [4/50], Step [240/469], Loss: 0.5056
Epoch [4/50], Step [260/469], Loss: 0.5447
Epoch [4/50], Step [280/469], Loss: 0.6448
Epoch [4/50], Step [300/469], Loss: 0.4839
Epoch [4/50], Step [320/469], Loss: 0.5054
Epoch [4/50], Step [340/469], Loss: 0.4640
Epoch [4/50], Step [360/469], Loss: 0.5055
Epoch [4/50], Step [380/469], Loss: 0.5448
Epoch [4/50], Step [400/469], L

Epoch [6/50], Step [340/469], Loss: 0.5254
Epoch [6/50], Step [360/469], Loss: 0.6308
Epoch [6/50], Step [380/469], Loss: 0.5254
Epoch [6/50], Step [400/469], Loss: 0.4864
Epoch [6/50], Step [420/469], Loss: 0.5054
Epoch [6/50], Step [440/469], Loss: 0.7060
Epoch [6/50], Step [460/469], Loss: 0.4854
Epoch [6/50], Loss: 0.5241
Epoch [7/50], Step [20/469], Loss: 0.5450
Epoch [7/50], Step [40/469], Loss: 0.4660
Epoch [7/50], Step [60/469], Loss: 0.4483
Epoch [7/50], Step [80/469], Loss: 0.5462
Epoch [7/50], Step [100/469], Loss: 0.4226
Epoch [7/50], Step [120/469], Loss: 0.5254
Epoch [7/50], Step [140/469], Loss: 0.5048
Epoch [7/50], Step [160/469], Loss: 0.5051
Epoch [7/50], Step [180/469], Loss: 0.6773
Epoch [7/50], Step [200/469], Loss: 0.4853
Epoch [7/50], Step [220/469], Loss: 0.5452
Epoch [7/50], Step [240/469], Loss: 0.4839
Epoch [7/50], Step [260/469], Loss: 0.5065
Epoch [7/50], Step [280/469], Loss: 0.5267
Epoch [7/50], Step [300/469], Loss: 0.5260
Epoch [7/50], Step [320/469], L

Epoch [9/50], Step [260/469], Loss: 0.5624
Epoch [9/50], Step [280/469], Loss: 0.5254
Epoch [9/50], Step [300/469], Loss: 0.5456
Epoch [9/50], Step [320/469], Loss: 0.6065
Epoch [9/50], Step [340/469], Loss: 0.5048
Epoch [9/50], Step [360/469], Loss: 0.4214
Epoch [9/50], Step [380/469], Loss: 0.5668
Epoch [9/50], Step [400/469], Loss: 0.4669
Epoch [9/50], Step [420/469], Loss: 0.5655
Epoch [9/50], Step [440/469], Loss: 0.6063
Epoch [9/50], Step [460/469], Loss: 0.5655
Epoch [9/50], Loss: 0.5239
Epoch [10/50], Step [20/469], Loss: 0.7125
Epoch [10/50], Step [40/469], Loss: 0.5459
Epoch [10/50], Step [60/469], Loss: 0.5454
Epoch [10/50], Step [80/469], Loss: 0.5630
Epoch [10/50], Step [100/469], Loss: 0.4883
Epoch [10/50], Step [120/469], Loss: 0.4523
Epoch [10/50], Step [140/469], Loss: 0.6947
Epoch [10/50], Step [160/469], Loss: 0.5650
Epoch [10/50], Step [180/469], Loss: 0.4846
Epoch [10/50], Step [200/469], Loss: 0.4842
Epoch [10/50], Step [220/469], Loss: 0.4442
Epoch [10/50], Step 

Epoch [12/50], Step [160/469], Loss: 0.5647
Epoch [12/50], Step [180/469], Loss: 0.5649
Epoch [12/50], Step [200/469], Loss: 0.3844
Epoch [12/50], Step [220/469], Loss: 0.5467
Epoch [12/50], Step [240/469], Loss: 0.5447
Epoch [12/50], Step [260/469], Loss: 0.5657
Epoch [12/50], Step [280/469], Loss: 0.4668
Epoch [12/50], Step [300/469], Loss: 0.5257
Epoch [12/50], Step [320/469], Loss: 0.5446
Epoch [12/50], Step [340/469], Loss: 0.5056
Epoch [12/50], Step [360/469], Loss: 0.5253


In [0]:
def infer(sens):
    
    outputs = model(sens[0], sens[1], [sens[0].size(1)], [sens[1].size(1)])  
    return np.argmax(outputs)

In [108]:
#test = random.choice(train_data)
#test = random.sample(train_data, 5)

sen1, sen2, targets,sen1_lengths, sen2_lengths = pad_to_batch(test,source2index)
outputs = model(sen1, sen2, sen1_lengths, sen2_lengths)

predict = np.array(np.argmax(outputs.data, 1))
targets = np.array(targets.data)
print("Predict:", predict)
print("Truth:", targets)
print("Correct: ", np.sum(predict == targets))


Predict: [0 0 0 0 0]
Truth: [0 1 0 0 0]
Correct:  4


In [0]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score


print("confusion_matrix: \n" ,confusion_matrix(predict, targets)
print(precision_score(predict, targets))
print(recall_score(predict, targets))
print(f1_score(predict, targets))

## tn fp
## fn tp